# Project 4 Project
## Recommendations of Movies

There are more than millions of movies made since first introducted but limited time  to see the movies people would like. In order to help consumers save time and money, recommendations are made to make selection easier. To make these recommendation, we will be using a dataset of roughly 10,000 movie entries inorder to predict that a user would rate a given movie higher with those with users with similar reviews on said movie.   

In [1]:
### Read all the dataset and load them in with proper names ###
### Links were not used for this project ###

import pandas as pd
movies = pd.read_csv('ml-latest-small/movies.csv')
# links = pd.read_csv('ml-latest-small/links.csv') 
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

### Exploration Information
* There are 9742 movies in the dataset
* There are 100836 ratings and 610 users 
* There are 58 users that make up the 3683 tags added
* Genres are separated by | if there are more than one

In [2]:
ratings.info()
ratings.drop('timestamp', axis = 1, inplace = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [3]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(ratings,reader)
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


In [4]:
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [5]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

In [6]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8698090142276478, 'mae': 0.6682616282789757}
{'rmse': {'n_factors': 20, 'reg_all': 0.02}, 'mae': {'n_factors': 20, 'reg_all': 0.02}}


In [7]:
svd = SVD(n_factors= 100, reg_all=0.05)
svd.fit(dataset)

In [8]:
movie_ids = movies['movieId'].unique()

In [31]:
### Function that takes in a user Id and recommends the top 5 best movies from the model ###
def users_top_5(user_id):
    movie_est = []
    for m_id in movie_ids:
        prediction = svd.predict(user_id, m_id)
        movie_est.append((prediction[1], prediction[3]))
   
    movie_est = sorted(movie_est, key= lambda x: x[1], reverse= True)
    top_5_movies = [movie[0] for movie in movie_est[0:5]]
    display(movies.loc[movies['movieId'].isin(top_5_movies)])
    

In [41]:
users_top_5()

TypeError: users_top_5() missing 1 required positional argument: 'user_id'